In [1]:
import numpy as np
import csv
import tensorflow as tf

from model import build_model_reddit, build_rnn_reddit
from config import reddit_num_input, reddit_learning_rate, reddit_num_classes

with open("bitstamp_day.csv",'r') as f:
    bitcoin_daily = []
    rdr = csv.reader(f)
    for line in rdr:
        bitcoin_daily.append(line)

bitcoin_daily = np.array(bitcoin_daily)

year = 2015
month = 1
lapl_mat_list = []
features_list = []
for i in range(reddit_num_input):
    day = i + 1
    with open("keyword/adj_{0}_{1}_{2}.csv".format(year,month,day),'r') as f:
        lapl_mat_temp = []
        rdr = csv.reader(f)
        for line in rdr:
            lapl_mat_temp.append(line)

    with open("keyword/features_{0}_{1}_{2}.csv".format(year,month,day),'r') as f:
        features_temp = []
        rdr = csv.reader(f)
        for line in rdr:
            features_temp.append(line)

    lapl_mat_temp = np.array(lapl_mat_temp) 
    features_temp = np.array(features_temp) 

    num_nodes = len(lapl_mat_temp)
    if i == 0:
        num_nodes_min = num_nodes
    
    # truncate adjacency_matrix & features
    lapl_mat_temp = lapl_mat_temp[0:num_nodes_min,0:num_nodes_min]
    features_temp = features_temp[0:num_nodes_min,:]
    
    lapl_mat = [None] * num_nodes_min
    features = [None] * num_nodes_min

    for ii in range(num_nodes_min):
        lapl_mat[ii] = list(map(int, lapl_mat_temp[ii]))
        features[ii] = list(map(float, features_temp[ii]))

    lapl_mat = lapl_mat + np.eye(num_nodes_min)

    features = np.array(features)

    batch_size = 1

    # bitcoin up/down
    # one-hot vector
    if i == reddit_num_input -1:
        
        ind_date = (bitcoin_daily[:,0]==str(year)) * (bitcoin_daily[:,1]==' '+str(month)) * (bitcoin_daily[:,2]==' '+str(day))
        up_down = float(bitcoin_daily[ind_date,4])
        label_for_rnn = np.zeros((batch_size, reddit_num_classes))
        if up_down == 1:
            label_for_rnn[:,0] = 1    
        elif up_down == -1:
            label_for_rnn[:,1] = 1    
        else:
            label_for_rnn[:,2] = 1

    print("lapl_mat", lapl_mat.shape)
    print("features", features.shape)
    #print("labels", labels.shape)
    
    lapl_mat_list.append(lapl_mat)
    features_list.append(features)
    

tf_config = tf.ConfigProto()
graph = tf.Graph()
with graph.as_default():
    
    plhdr_label = tf.placeholder(tf.float32, 
                                 shape=(None, reddit_num_classes),
                                 name='label')
    
    embedded_feature_list = build_model_reddit()
    rnn_output = build_rnn_reddit(embedded_feature_list)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=rnn_output, labels=plhdr_label))
    train = tf.train.AdamOptimizer(reddit_learning_rate).minimize(cross_entropy)
    
    prediction = tf.argmax(rnn_output, 1)
    correction = tf.argmax(label_for_rnn, 1)

    correct_prediction = tf.cast(tf.equal(prediction, correction), tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)
    
    
    
with tf.Session(graph=graph, config=tf_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    # Generate fake feed_dict for list of input
    feed_dict = dict()
    feed_dict['label:0'] = label_for_rnn
    for i in range(reddit_num_input):
        feed_dict['input_{}:0'.format(i)] = features_list[i]
        feed_dict['adj_{}:0'.format(i)] = lapl_mat_list[i]
    
    _embedded_feature_list = sess.run(embedded_feature_list, feed_dict=feed_dict)
    _rnn_logits = sess.run(rnn_output, feed_dict=feed_dict)
    
    print(_embedded_feature_list.shape, _embedded_feature_list[0], _rnn_logits.shape)
    
    num_epoch = 200
    for epoch in range(num_epoch):
        sess.run(train, feed_dict=feed_dict)
        if (1+epoch)%10 == 0:
            _pred, _acc, _loss = sess.run([prediction, accuracy, cross_entropy], feed_dict=feed_dict)
            print("="*80)
            print("Epoch -", epoch, "label -", _pred[0], "loss -", _loss)
    _pred, _acc, _loss = sess.run([prediction, accuracy, cross_entropy], feed_dict=feed_dict)
    print("="*80)
    print("Epoch -", 199, "label -", _pred[0], "loss -", _loss)
    

lapl_mat (982, 982)
features (982, 7)
lapl_mat (982, 982)
features (982, 7)
lapl_mat (982, 982)
features (982, 7)
lapl_mat (982, 982)
features (982, 7)
lapl_mat (982, 982)
features (982, 7)
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

(5, 7) [ 6.10799120e+07  2.21012980e+07 -4.72783000e+07 -1.06666216e+08
 -2.13608220e+07 -2.36767400e+07  1.11154910e+07] (1, 3)
Epoch - 9 label - 0 loss - 0.12635228
Epoch - 19 label - 0 loss - 0.08907665
Epoch - 29 label - 0 loss - 0.06508303
Epoch - 39 label - 0 loss - 0.049600072
Epoch - 49 label - 0 loss - 0.039314777
Epoch - 59 label - 0 loss - 0.032193992
Epoch - 69 label - 0 l